In [1]:
%env TORCH_SHOW_DISPATCH_TRACE=1

env: TORCH_SHOW_DISPATCH_TRACE=1


In [2]:
import torch
import torch_xla
import torch_xla.runtime
import time

time.sleep(5)

 [call] op=[aten::ones], key=[BackendSelect]
  [redispatch] op=[aten::ones], key=[CPU]
   [call] op=[aten::empty.memory_format], key=[BackendSelect]
    [redispatch] op=[aten::empty.memory_format], key=[CPU]
   [call] op=[aten::fill_.Scalar], key=[CPU]
 [call] op=[aten::ones], key=[BackendSelect]
  [redispatch] op=[aten::ones], key=[CPU]
   [call] op=[aten::empty.memory_format], key=[BackendSelect]
    [redispatch] op=[aten::empty.memory_format], key=[CPU]
   [call] op=[aten::fill_.Scalar], key=[CPU]
 [call] op=[aten::ones], key=[BackendSelect]
  [redispatch] op=[aten::ones], key=[CPU]
   [call] op=[aten::empty.memory_format], key=[BackendSelect]
    [redispatch] op=[aten::empty.memory_format], key=[CPU]
   [call] op=[aten::fill_.Scalar], key=[CPU]
 [call] op=[aten::ones], key=[BackendSelect]
  [redispatch] op=[aten::ones], key=[CPU]
   [call] op=[aten::empty.memory_format], key=[BackendSelect]
    [redispatch] op=[aten::empty.memory_format], key=[CPU]
   [call] op=[aten::fill_.Scalar]

torch.einsum turns into an XLA DotGeneral

In [3]:
with torch_xla.runtime.xla_device():
  a = torch.rand((40, 30, 20), dtype=torch.bfloat16)
  b = torch.rand((15, 20), dtype=torch.bfloat16)
  torch_xla.sync()
  _ = str(a), str(b)  # workaround _get_xla_tensors_hlo crash
  c = torch.einsum('...n,mn->...m', a, b)

time.sleep(5)
print(torch_xla._XLAC._get_xla_tensors_hlo([c]))

 [call] op=[aten::rand], key=[BackendSelect]
  [redispatch] op=[aten::rand], key=[XLA]
   [call] op=[aten::empty.memory_format], key=[BackendSelect]
    [redispatch] op=[aten::empty.memory_format], key=[XLA]
   [call] op=[aten::uniform_], key=[Functionalize]
    [call] op=[aten::uniform_], key=[Meta]
     [callBoxed] op=[prims::uniform], key=[BackendSelect]
      [call] op=[aten::empty_strided], key=[BackendSelect]
       [redispatch] op=[aten::empty_strided], key=[Meta]
     [call] op=[aten::copy_], key=[Meta]
    [call] op=[aten::uniform], key=[XLA]
     [call] op=[aten::clone], key=[XLA]
     [call] op=[aten::uniform_], key=[XLA]
      [call] op=[aten::scalar_tensor], key=[BackendSelect]
       [redispatch] op=[aten::scalar_tensor], key=[CPU]
      [call] op=[aten::to.dtype_layout], key=[BackendSelect]
       [redispatch] op=[aten::to.dtype_layout], key=[CPU]
        [call] op=[aten::_to_copy], key=[BackendSelect]
         [redispatch] op=[aten::_to_copy], key=[CPU]
          [call]

HloModule IrToHlo.6, entry_computation_layout={(bf16[15,20]{1,0}, bf16[40,30,20]{0,1,2})->(bf16[40,30,15]{2,1,0})}

ENTRY %IrToHlo.6 (p0.1: bf16[15,20], p1.2: bf16[40,30,20]) -> (bf16[40,30,15]) {
  %p1.2 = bf16[40,30,20]{0,1,2} parameter(1)
  %p0.1 = bf16[15,20]{1,0} parameter(0)
  %dot.3 = bf16[40,30,15]{2,1,0} dot(bf16[40,30,20]{0,1,2} %p1.2, bf16[15,20]{1,0} %p0.1), lhs_contracting_dims={2}, rhs_contracting_dims={1}, frontend_attributes={grad_x="false",grad_y="false"}
  %transpose.4 = bf16[40,30,15]{2,1,0} transpose(bf16[40,30,15]{2,1,0} %dot.3), dimensions={0,1,2}
  ROOT %tuple.5 = (bf16[40,30,15]{2,1,0}) tuple(bf16[40,30,15]{2,1,0} %transpose.4)
}




torch.nn.functional.linear also turns into an XLA DotGeneral, but with flattened inputs.

In [4]:
with torch_xla.runtime.xla_device():
  a = torch.rand((40, 30, 20), dtype=torch.bfloat16)
  b = torch.rand((15, 20), dtype=torch.bfloat16)
  torch_xla.sync()
  _ = str(a), str(b)  # workaround _get_xla_tensors_hlo crash
  c = torch.nn.functional.linear(a, b)

time.sleep(5)
print(torch_xla._XLAC._get_xla_tensors_hlo([c]))

 [call] op=[aten::rand], key=[BackendSelect]
  [redispatch] op=[aten::rand], key=[XLA]
   [call] op=[aten::empty.memory_format], key=[BackendSelect]
    [redispatch] op=[aten::empty.memory_format], key=[XLA]
   [call] op=[aten::uniform_], key=[Functionalize]
    [call] op=[aten::uniform_], key=[Meta]
     [callBoxed] op=[prims::uniform], key=[BackendSelect]
      [call] op=[aten::empty_strided], key=[BackendSelect]
       [redispatch] op=[aten::empty_strided], key=[Meta]
     [call] op=[aten::copy_], key=[Meta]
    [call] op=[aten::uniform], key=[XLA]
     [call] op=[aten::clone], key=[XLA]
     [call] op=[aten::uniform_], key=[XLA]
      [call] op=[aten::scalar_tensor], key=[BackendSelect]
       [redispatch] op=[aten::scalar_tensor], key=[CPU]
      [call] op=[aten::to.dtype_layout], key=[BackendSelect]
       [redispatch] op=[aten::to.dtype_layout], key=[CPU]
        [call] op=[aten::_to_copy], key=[BackendSelect]
         [redispatch] op=[aten::_to_copy], key=[CPU]
          [call]

HloModule IrToHlo.8, entry_computation_layout={(bf16[15,20]{1,0}, bf16[40,30,20]{0,1,2})->(bf16[40,30,15]{2,1,0})}

ENTRY %IrToHlo.8 (p0.1: bf16[15,20], p1.3: bf16[40,30,20]) -> (bf16[40,30,15]) {
  %p1.3 = bf16[40,30,20]{0,1,2} parameter(1)
  %reshape.4 = bf16[1200,20]{1,0} reshape(bf16[40,30,20]{0,1,2} %p1.3)
  %p0.1 = bf16[15,20]{1,0} parameter(0)
  %transpose.2 = bf16[20,15]{0,1} transpose(bf16[15,20]{1,0} %p0.1), dimensions={1,0}
  %dot.5 = bf16[1200,15]{1,0} dot(bf16[1200,20]{1,0} %reshape.4, bf16[20,15]{0,1} %transpose.2), lhs_contracting_dims={1}, rhs_contracting_dims={0}
  %reshape.6 = bf16[40,30,15]{2,1,0} reshape(bf16[1200,15]{1,0} %dot.5)
  ROOT %tuple.7 = (bf16[40,30,15]{2,1,0}) tuple(bf16[40,30,15]{2,1,0} %reshape.6)
}


